In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from env import user, password, host
import wrangle_zillow


import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')


In [2]:
county_df = wrangle_zillow.wrangle_county_fips()

In [3]:
county_df

,County
fips_number,
6037,Los Angeles County
6059,Orange County
6111,Ventura County


In [12]:
zillow_df = wrangle_zillow.wrangle_zillow_data()

In [13]:
zillow_df.head()

,bedrooms,bathrooms,square_feet,property_tax,house_value,property_description,property_id,fips
0,4.0,2.0,1604.0,6089.82,498347.0,Single Family Residential,261.0,Los Angeles County
1,3.0,3.0,2384.0,6679.55,549917.0,Single Family Residential,261.0,Los Angeles County
2,3.0,2.0,1574.0,3876.31,235272.0,Single Family Residential,261.0,Los Angeles County
3,2.0,2.0,1619.0,4206.15,340000.0,Single Family Residential,261.0,Los Angeles County
4,2.0,3.0,2408.0,24353.42,2017254.0,Single Family Residential,261.0,Los Angeles County


In [16]:
zillow_df['property_tax_rate'] = zillow_df.property_tax / zillow_df.house_value

In [17]:
zillow_df.head()

,bedrooms,bathrooms,square_feet,property_tax,house_value,property_description,property_id,fips,property_tax_rate
0,4.0,2.0,1604.0,6089.82,498347.0,Single Family Residential,261.0,Los Angeles County,0.012220
1,3.0,3.0,2384.0,6679.55,549917.0,Single Family Residential,261.0,Los Angeles County,0.012146
2,3.0,2.0,1574.0,3876.31,235272.0,Single Family Residential,261.0,Los Angeles County,0.016476
3,2.0,2.0,1619.0,4206.15,340000.0,Single Family Residential,261.0,Los Angeles County,0.012371
4,2.0,3.0,2408.0,24353.42,2017254.0,Single Family Residential,261.0,Los Angeles County,0.012073


In [27]:
zillow_df.fips == 'Los Angeles County'

0         True
1         True
2         True
3         True
4         True
6         True
7         True
8         True
9         True
10        True
11        True
12        True
13        True
14        True
15        True
16        True
17        True
18        True
19        True
20        True
21        True
22        True
23        True
24        True
25        True
26        True
27        True
28        True
29        True
30        True
         ...  
16045    False
16046    False
16047    False
16048    False
16049    False
16050    False
16051    False
16052    False
16053    False
16054    False
16055    False
16056    False
16057    False
16058    False
16059    False
16060    False
16061    False
16062    False
16063    False
16064    False
16065    False
16066    False
16067    False
16068    False
16069    False
16070    False
16071    False
16072    False
16073    False
16074    False
Name: fips, Length: 16004, dtype: bool